In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.chdir('/content/gdrive/')
os.getcwd()

In [3]:
path = '/content/gdrive/MyDrive/'
files = os.listdir(path)

In [4]:
import numpy as np
import os
import time
import pickle
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras import backend as K
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize
from sklearn.svm import SVC

In [5]:
labels = []
feature_list = []
feature_dict=pickle.load(open('/content/gdrive/MyDrive/features_till_flatten.pkl','rb'))
names = [0,1]
for cat,features  in feature_dict.items():
    labels.extend([int(cat)] * len(features))
    feature_list.append(features)
feature_list = np.concatenate(feature_list)

In [6]:
feature_list1=[]
for i in range(0,len(feature_list)):
    feature_list1.append(feature_list[i].ravel())

In [ ]:
feature_list1[0].shape

In [ ]:
scaler = StandardScaler().fit(feature_list1)
rescaledX = scaler.transform(feature_list1)
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
model = SVC()
results = cross_val_score(model, rescaledX, labels)
print ("Accuracy: {} ({})".format(results.mean()*100.0, results.std()*100.0))

In [ ]:
scoring = 'accuracy'

c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = [ 'linear' , 'poly' , 'rbf' , 'sigmoid' ]
param_grid = dict(C=c_values, kernel=kernel_values)

model = SVC()

kfold = KFold(n_splits=num_folds, random_state=seed, shuffle= True)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)

grid_result = grid.fit(rescaledX, labels)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [10]:
tuned_model = SVC(C=0.1, kernel='rbf')

In [ ]:
tuned_model.fit(rescaledX, labels)

In [12]:
labels2 = []
feature_list = []
feature_dict=pickle.load(open('/content/gdrive/MyDrive/features_till_flatten_validation.pkl','rb'))
names = [0,1]
for cat,features  in feature_dict.items():
    labels2.extend([int(cat)] * len(features))
    feature_list.append(features)
feature_list = np.concatenate(feature_list)

In [13]:
feature_list2=[]
for i in range(0,len(feature_list)):
    feature_list2.append(feature_list[i].ravel())

In [ ]:
rescaledX_validation = scaler.transform(feature_list2)

result = tuned_model.score(rescaledX_validation, labels2)
print('Accuracy: %.3f%%' % (result * 100.0))